#### Framework de Big Data

#### Conteúdo - Bases e Notebook da aula

Base disponivel no caminho:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Framework-de-Big-Data

E para correto funcionamento foi necessario a criação de um roteiro de instalação e configuração: 

https://github.com/RicardViana/fiap-Big-Data/blob/main/PySpark%20para%20VS%20Code%20e%20Anaconda.pdf

#### Importação de pacotes, bibliotecas e funções (def)

In [96]:
# Importar biblioteca completa
import pandas as pd 
import findspark
import urllib.request

# Importar função especifica de um módulo
from pyspark.sql import SparkSession

# Import spark libraries
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F
from functools import reduce

In [97]:
# Como não queremos configurar as variaveis de ambiente, precisamos usar o findspark.init
findspark.init()

#### Aula 1 - Conhecendo o Spark

In [98]:
"""

1. spark = SparkSession.builder.master("local[*]").getOrCreate()
Esta é a linha que inicializa o Spark. Ela é o ponto de entrada para qualquer funcionalidade do Spark.

SparkSession: É o principal ponto de entrada para a programação com DataFrames e Datasets no Spark.

.builder: É um padrão de projeto (builder pattern) usado para construir o objeto SparkSession com diferentes configurações.

.master("local[*]"): Esta é uma das configurações mais importantes. Ela define como e onde o Spark irá executar suas tarefas.

"local[*]" instrui o Spark a ser executado em modo local, utilizando todos os núcleos de processamento (cores) disponíveis na sua máquina. 
Se você quisesse usar apenas 2 núcleos, por exemplo, usaria "local[2]". Este modo é ideal para desenvolvimento, testes e aprendizado em uma única máquina.

.getOrCreate(): Este método cria uma nova SparkSession se uma não existir. Se já houver uma sessão ativa com as mesmas configurações, ele simplesmente a retorna. 
Isso evita a criação de múltiplas sessões desnecessariamente.

Em resumo: esta linha cria e configura uma sessão do Spark para rodar localmente na sua máquina, usando todos os recursos de processamento disponíveis, e armazena essa sessão na variável spark.

2. df = spark.read.csv(link, sep= ",", inferSchema = True, header = True)
Esta é a linha que efetivamente lê os dados e os transforma em um DataFrame, que é a principal estrutura de dados do Spark.

spark.read: É o objeto usado para ler dados de fontes externas (arquivos, bancos de dados, etc.).

.csv(link, ...): Especifica que o formato do arquivo a ser lido é CSV. O primeiro argumento, link, é o caminho para o arquivo.

sep= ",": Este parâmetro (separador) informa ao Spark que as colunas no arquivo CSV são delimitadas por uma vírgula. É o padrão para arquivos CSV, mas é uma boa prática especificá-lo.

inferSchema = True: Esta é uma opção muito útil. Ela instrui o Spark a analisar uma amostra dos dados para inferir (deduzir) automaticamente o tipo de dado de cada coluna. 
Por exemplo, ele tentará identificar se uma coluna contém números inteiros (IntegerType), números decimais (DoubleType) ou texto (StringType). Sem isso, todas as colunas seriam tratadas como texto (String).

header = True: Esta opção informa ao Spark que a primeira linha do arquivo CSV contém os nomes das colunas (cabeçalho) e que essa linha não deve ser tratada como dados. 
O Spark usará esses nomes para as colunas do DataFrame.

"""

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Baixar o arquivo pois não conseguimos usar o Spark para ler diretamnte o arquivo no Github
url = "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/Framework-de-Big-Data/Aula%201/banklist.csv"
local_path = "banklist.csv"
urllib.request.urlretrieve(url, local_path)

# Ler com Spark
df = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)

print(f"df.count: {df.count()}")
print(f"df.col ct: {len(df.columns)}")
print(f"df.columns: {df.columns}")

df.count: 561
df.col ct: 6
df.columns: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


In [99]:
# Criar uma tabela temporaria
df.createOrReplaceTempView("banklist")

# Fazer o select 
df_check = spark.sql(''' select `Bank Name`, `City`,`Closing Date` from banklist ''')
df_check.show(4, truncate=False) # --> Retornar apenas os 4 primeiros dados e não truncar textos longos

+--------------------------------+-------------+------------+
|Bank Name                       |City         |Closing Date|
+--------------------------------+-------------+------------+
|The First State Bank            |Barboursville|3-Apr-20    |
|Ericson State Bank              |Ericson      |14-Feb-20   |
|City National Bank of New Jersey|Newark       |1-Nov-19    |
|Resolute Bank                   |Maumee       |25-Oct-19   |
+--------------------------------+-------------+------------+
only showing top 4 rows


In [100]:
# Executar a estatistica descritiva 
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [101]:
# Executar a estatistica descritiva 
df.describe('City', 'ST').show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



In [102]:
# Ver informações do data frame 
print(f"Total de linhas: {df.count()}")
print(f"Total de colunas: {len(df.columns)}")
print(f"Colunas: {df.columns}")
print(f"Tipo de dados: {df.dtypes}")
print(f"Schema {df.schema}")


Total de linhas: 561
Total de colunas: 6
Colunas: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipo de dados: [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


In [103]:
# Ver o Schema dos dados 
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



In [104]:
# Remover duplicadas 
df = df.dropDuplicates()
print(f"df.count: {df.count()}")
print(f"df.columns: {df.columns}")

df.count: 561
df.columns: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


In [105]:
# Selecionar colunas especificas do Data Set
df2 = df.select(*['Bank Name', 'City'])
df2.show(truncate=False)

+---------------------------------+----------------+
|Bank Name                        |City            |
+---------------------------------+----------------+
|First Bank of Idaho              |Ketchum         |
|Amcore Bank, National Association|Rockford        |
|Venture Bank                     |Lacey           |
|First State Bank of Altus        |Altus           |
|Valley Capital Bank, N.A.        |Mesa            |
|Michigan Heritage Bank           |Farmington Hills|
|Columbia Savings Bank            |Cincinnati      |
|Fidelity Bank                    |Dearborn        |
|The Park Avenue Bank             |Valdosta        |
|Western Commercial Bank          |Woodland Hills  |
|Syringa Bank                     |Boise           |
|Republic Federal Bank, N.A.      |Miami           |
|Westside Community Bank          |University Place|
|First United Bank                |Crete           |
|HarVest Bank of Maryland         |Gaithersburg    |
|BankEast                         |Knoxville  

In [106]:
# Selecionar colunas exceto o que tiver na lista 
col_l = list(set(df.columns) - {'ST', 'CERT'})
df2 = df.select(*col_l)
df2.show(truncate=False)

+---------------------------------+----------------------------------------------------+------------+----------------+
|Bank Name                        |Acquiring Institution                               |Closing Date|City            |
+---------------------------------+----------------------------------------------------+------------+----------------+
|First Bank of Idaho              |U.S. Bank, N.A.                                     |24-Apr-09   |Ketchum         |
|Amcore Bank, National Association|Harris N.A.                                         |23-Apr-10   |Rockford        |
|Venture Bank                     |First-Citizens Bank & Trust Company                 |11-Sep-09   |Lacey           |
|First State Bank of Altus        |Herring Bank                                        |31-Jul-09   |Altus           |
|Valley Capital Bank, N.A.        |Enterprise Bank & Trust                             |11-Dec-09   |Mesa            |
|Michigan Heritage Bank           |Level One Ban

In [107]:
# Renomear as colunas 
df2 = df \
  .withColumnRenamed('Bank Name'            , 'bank_name') \
  .withColumnRenamed('Acquiring Institution', 'acq_institution') \
  .withColumnRenamed('Closing Date'         , 'closing_date') \
  .withColumnRenamed('ST'                   , 'state') \
  .withColumnRenamed('CERT'                 , 'cert') 

df2.show()

+--------------------+----------------+-----+-----+--------------------+------------+
|           bank_name|            City|state| cert|     acq_institution|closing_date|
+--------------------+----------------+-----+-----+--------------------+------------+
| First Bank of Idaho|         Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|   MI|34369|      Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati|   OH|32284|United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn|   MI|33883|The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   GA|19797|  B

In [108]:
# Adicionar uma nova coluna 
df2 = df.withColumn('State', col('ST'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-----+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|State|
+--------------------+--------+---+-----+---------------------+------------+-----+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|   ID|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|   IL|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|   WA|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|   OK|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|   AZ|
+--------------------+--------+---+-----+---------------------+------------+-----+
only showing top 5 rows


In [109]:
# Adicionar uma nova coluna
df2 = df.withColumn('country', lit('US'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+--------+---+-----+---------------------+------------+-------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|     US|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|     US|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|     US|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|     US|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|     US|
+--------------------+--------+---+-----+---------------------+------------+-------+
only showing top 5 rows


In [110]:
# Apagar coluna 
df2 = df.drop('CERT')
df2.show(5)

+--------------------+--------+---+---------------------+------------+
|           Bank Name|    City| ST|Acquiring Institution|Closing Date|
+--------------------+--------+---+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| WA| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus| OK|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| AZ| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---+---------------------+------------+
only showing top 5 rows


In [111]:
# Apagar varias colunas 
df2 = df.drop(*["CERT","ST"])
df2.show(5)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---------------------+------------+
only showing top 5 rows


In [112]:
# Apagar varias colunas com reduce 
df2 = reduce(DataFrame.drop, ["CERT","ST"], df)
df2.show(5)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---------------------+------------+
only showing top 5 rows


In [113]:
# Realizar filtros 
df2 = df.where(df['ST'] == 'NE')
df3 = df.where(df['CERT'].between('1000','2000'))
df4 = df.where(df['ST'].isin('NE','IL'))

print('df.count  :', df.count())
print('df2.count :', df2.count())
print('df3.count :', df3.count())
print('df4.count :', df4.count())


df.count  : 561
df2.count : 4
df3.count : 9
df4.count : 73


In [114]:
# Realizar filtros --> AND
df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson'))
df2.show(3)

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



In [ ]:
# Alterar os dados --> Replace
df.show(5)

print('Replace 7 in the above dataframe with 17 at all instances')
df.na.replace(7,17).show(5) # --> Dessa forma, substitui em todas as colunas o que for 7 por 17

+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---+-----+---------------------+------------+
only showing top 5 rows
Replace 7 in the above dataframe with 17 at all instances
+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+--------

#### Aula 2 - Operações Básicas no Spark

#### Aula 3 - Consultas e Seleções

#### Aula 4 - Operações entre Dataframes e Armazenamento

#### Aula 5 - Introdução aos Sistemas de Recomendação

#### Aula 6 - Recomendações com o Algoritmo ALS